## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
sample_no = 3
data_dir = f"../data/PubMed/sample{sample_no}/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_11.pkl",'rb'))
U2 = pkl.load(open(data_dir+"X_22.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_31.pkl",'rb'))
V2 = pkl.load(open(data_dir+"X_32.pkl",'rb'))
V3 = pkl.load(open(data_dir+"X_33.pkl",'rb'))
V4 = pkl.load(open(data_dir+"X_34.pkl",'rb'))
W1 = pkl.load(open(data_dir+"X_41.pkl",'rb'))
W2 = pkl.load(open(data_dir+"X_42.pkl",'rb'))
W3 = pkl.load(open(data_dir+"X_44.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_12_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_12_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_12_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_12_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"U2":U2,"V1":V1,"V2":V2,"V3":V3,"V4":V4,"W1":W1,"W2":W2,"W3":W3,"R":R_temp_dict}

Loading data from data_dir:  ../data/PubMed/sample3/


In [6]:
print("U1.shape: ",U1.shape)
print("U1.shape: ",U2.shape)
print("V1.shape: ",V1.shape)
print("V2.shape: ",V2.shape)
print("V3.shape: ",V3.shape)
print("V4.shape: ",V4.shape)
print("W1.shape: ",W1.shape)
print("W2.shape: ",W2.shape)
print("W3.shape: ",W3.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (2665, 2665)
U1.shape:  (4454, 4454)
V1.shape:  (5877, 2665)
V2.shape:  (5877, 4454)
V3.shape:  (5877, 5877)
V4.shape:  (5877, 619)
W1.shape:  (619, 2665)
W2.shape:  (619, 4454)
W3.shape:  (619, 619)
R.shape:  (2665, 4454)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X2","X4","X8"],\
    "e2":["X2","X3","X5","X9"],\
    "e3":["X4","X5","X6","X7"],\
    "e4":["X7","X8","X9","X10"]
}
    #"e6":["X4"]}

In [8]:
X_data = {
    "X1":{"1":U1},\
    "X2":{"1":data_dict['R'][1]["Rtrain"]},\
    "X3":U2,\
    "X4":V1,\
    "X5":V2,\
    "X6":V3,\
    "X7":V4,\
    "X8":W1,\
    "X9":W2,\
    "X10":W3
}

In [9]:
X_meta = {
    "X1":["e1","e1"],\
    "X2":["e1","e2"],\
    "X3":["e2","e2"],\
    "X4":["e3","e1"],\
    "X5":["e3","e2"],\
    "X6":["e3","e3"],\
    "X7":["e3","e4"],\
    "X8":["e4","e1"],\
    "X9":["e4","e2"],\
    "X10":["e4","e4"]
}
    #"X5":["e5","e3"]}

In [10]:
# Rtest_triplets1 = [[1,1,1],[2,2,0]]
# Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]
Rtest_triplets1 = [[57, 1769, 0], [1708, 538, 1]]
Rtest_triplets2 = [[360, 2032, 1], [2506, 2430, 0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  4
num matrices:  10
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  4
num matrices:  10
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X2', 'X4', 'X8']
X_id:  X1
X[X_id].shape:  (2665, 2665)
X_id:  X2
X[X_id].shape:  (2665, 4454)
X_id:  X4
X[X_id].shape:  (5877, 2665)
X_id:  X8
X[X_id].shape:  (619, 2665)
C_dict[e].shape:  torch.Size([2665, 13615])
---
e_id:  e2
X_id_list:  ['X2', 'X3', 'X5', 'X9']
X_

#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [19]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2', 'e3', 'e4'])

In [20]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10'])

In [21]:
dcmf_model.out_dict_info

{'params': {'learning_rate': 0.001,
  'weight_decay': 0.05,
  'convg_thres': 0.1,
  'max_epochs': 5,
  'is_pretrain': True,
  'pretrain_thres': 0.1,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'k': 100,
  'kf': 0.5,
  'e_actf': 'tanh',
  'd_actf': 'tanh',
  'is_linear_last_enc_layer': False,
  'is_linear_last_dec_layer': False},
 'num_val_sets': 1,
 'loss_all_folds': {'1': [1.3187314569950104,
   0.7289924621582031,
   0.5856656432151794,
   1.1403934806585312,
   0.0562425721436739,
   0.8127592345699668,
   0.17208244558423758,
   1.9155877232551575,
   0.8732986841350794,
   0.5460205301642418,
   0.02135323639959097,
   10.097128895809874,
   0.19491947907954454,
   0.041732849553227425]},
 'loss_all_folds_avg_tuple': [1.3187314569950104,
  0.7289924621582031,
  0.5856656432151794,
  1.1403934806585312,
  0.0562425721436739,
  0.8127592345699668,
  0.17208244558423758,
  1.9155877232551575,
  0.8732986841350794,
  0.5460205301642418,
  0.02135323639959097,
  10.097128895809874

In [22]:
gene_emb = list(dcmf_model.out_dict_U['1'].values())[0].detach().numpy()
disease_emb = list(dcmf_model.out_dict_U['1'].values())[1].detach().numpy()
chemical_emb = list(dcmf_model.out_dict_U['1'].values())[2].detach().numpy()
species_emb = list(dcmf_model.out_dict_U['1'].values())[3].detach().numpy()

In [23]:
import pandas as pd

In [24]:
gene_emb_df = pd.DataFrame(gene_emb)
gene_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.055339,-0.016759,-0.039913,-0.020753,-0.004650,-0.020948,-0.025554,-0.003478,0.020021,-0.034456,...,-0.008454,-0.032839,0.004537,-0.020350,0.014823,0.054477,-0.007110,-0.021628,0.046493,-0.037668
1,0.055339,-0.016759,-0.039914,-0.020752,-0.004650,-0.020947,-0.025553,-0.003478,0.020021,-0.034455,...,-0.008454,-0.032839,0.004537,-0.020350,0.014824,0.054476,-0.007110,-0.021628,0.046494,-0.037668
2,0.055339,-0.016759,-0.039914,-0.020753,-0.004650,-0.020947,-0.025554,-0.003478,0.020020,-0.034456,...,-0.008454,-0.032839,0.004537,-0.020350,0.014824,0.054476,-0.007110,-0.021628,0.046494,-0.037668
3,0.055339,-0.016759,-0.039913,-0.020752,-0.004650,-0.020947,-0.025554,-0.003478,0.020020,-0.034456,...,-0.008453,-0.032840,0.004537,-0.020350,0.014824,0.054476,-0.007110,-0.021628,0.046494,-0.037668
4,0.055328,-0.016759,-0.039909,-0.020744,-0.004655,-0.020945,-0.025552,-0.003484,0.020014,-0.034450,...,-0.008452,-0.032841,0.004530,-0.020343,0.014825,0.054468,-0.007115,-0.021629,0.046495,-0.037670


In [25]:
gene_emb_df.shape

(2665, 100)

In [26]:
disease_emb_df = pd.DataFrame(disease_emb)
disease_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.007626,0.020096,0.005003,0.006253,0.021716,-0.002485,-0.010602,-0.021828,0.023859,-0.045248,...,0.044116,0.005015,-0.034266,-0.043137,-0.013076,0.032979,0.046098,0.049909,-0.003821,0.013142
1,-0.007626,0.020096,0.005002,0.006253,0.021716,-0.002485,-0.010602,-0.021828,0.023860,-0.045248,...,0.044116,0.005015,-0.034265,-0.043137,-0.013077,0.032978,0.046098,0.049908,-0.003821,0.013142
2,-0.007626,0.020096,0.005003,0.006253,0.021716,-0.002485,-0.010602,-0.021828,0.023860,-0.045248,...,0.044116,0.005015,-0.034265,-0.043138,-0.013077,0.032979,0.046098,0.049909,-0.003821,0.013142
3,-0.007626,0.020096,0.005002,0.006253,0.021716,-0.002485,-0.010602,-0.021828,0.023860,-0.045248,...,0.044116,0.005015,-0.034266,-0.043138,-0.013076,0.032979,0.046098,0.049908,-0.003821,0.013142
4,-0.007625,0.020097,0.005002,0.006253,0.021715,-0.002485,-0.010602,-0.021828,0.023859,-0.045248,...,0.044116,0.005015,-0.034265,-0.043137,-0.013076,0.032978,0.046098,0.049908,-0.003821,0.013141


In [27]:
disease_emb_df.shape

(4454, 100)

In [28]:
chemical_emb_df = pd.DataFrame(chemical_emb)
chemical_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.033600,-0.041082,-0.037530,0.005493,-0.016622,-0.015053,0.039332,-0.004111,-0.031153,0.017506,...,0.007579,-0.046627,-0.004532,0.080414,0.057859,0.056795,0.022736,-0.019172,-0.044943,-0.044370
1,-0.033597,-0.041076,-0.037530,0.005497,-0.016621,-0.015050,0.039328,-0.004113,-0.031149,0.017508,...,0.007578,-0.046625,-0.004534,0.080412,0.057861,0.056793,0.022734,-0.019173,-0.044941,-0.044365
2,-0.033594,-0.041071,-0.037529,0.005499,-0.016621,-0.015049,0.039325,-0.004115,-0.031146,0.017509,...,0.007579,-0.046623,-0.004535,0.080411,0.057863,0.056792,0.022733,-0.019174,-0.044940,-0.044363
3,-0.033601,-0.041085,-0.037530,0.005492,-0.016622,-0.015055,0.039334,-0.004110,-0.031155,0.017506,...,0.007579,-0.046628,-0.004532,0.080415,0.057858,0.056796,0.022738,-0.019172,-0.044944,-0.044372
4,-0.033602,-0.041085,-0.037530,0.005492,-0.016623,-0.015055,0.039333,-0.004110,-0.031154,0.017506,...,0.007579,-0.046629,-0.004532,0.080415,0.057858,0.056796,0.022738,-0.019172,-0.044944,-0.044372


In [29]:
chemical_emb_df.shape

(5877, 100)

In [30]:
species_emb_df = pd.DataFrame(species_emb)
species_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.020714,0.056579,0.012491,0.004107,-0.062899,0.044630,0.025475,0.009678,-0.009019,-0.003870,...,0.028161,0.036617,-0.031414,-0.029266,0.057714,0.027240,0.045755,-0.042297,0.012902,-0.014757
1,-0.020715,0.056579,0.012492,0.004105,-0.062897,0.044632,0.025475,0.009680,-0.009022,-0.003872,...,0.028157,0.036614,-0.031417,-0.029267,0.057717,0.027237,0.045754,-0.042295,0.012902,-0.014760
2,-0.020714,0.056578,0.012491,0.004106,-0.062898,0.044628,0.025473,0.009679,-0.009020,-0.003870,...,0.028160,0.036618,-0.031415,-0.029267,0.057711,0.027240,0.045752,-0.042296,0.012901,-0.014757
3,-0.020715,0.056578,0.012492,0.004105,-0.062897,0.044632,0.025474,0.009680,-0.009023,-0.003872,...,0.028157,0.036614,-0.031417,-0.029268,0.057717,0.027237,0.045753,-0.042296,0.012902,-0.014761
4,-0.020715,0.056579,0.012492,0.004105,-0.062897,0.044633,0.025475,0.009680,-0.009023,-0.003872,...,0.028157,0.036614,-0.031417,-0.029267,0.057717,0.027237,0.045753,-0.042295,0.012902,-0.014761


In [31]:
species_emb_df.shape

(619, 100)

In [32]:
emb_df = pd.concat([gene_emb_df, disease_emb_df, chemical_emb_df, species_emb_df], ignore_index = True, axis = 0)
emb_df.shape

(13615, 100)

In [33]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.055339,-0.016759,-0.039913,-0.020753,-0.004650,-0.020948,-0.025554,-0.003478,0.020021,-0.034456,...,-0.008454,-0.032839,0.004537,-0.020350,0.014823,0.054477,-0.007110,-0.021628,0.046493,-0.037668
1,0.055339,-0.016759,-0.039914,-0.020752,-0.004650,-0.020947,-0.025553,-0.003478,0.020021,-0.034455,...,-0.008454,-0.032839,0.004537,-0.020350,0.014824,0.054476,-0.007110,-0.021628,0.046494,-0.037668
2,0.055339,-0.016759,-0.039914,-0.020753,-0.004650,-0.020947,-0.025554,-0.003478,0.020020,-0.034456,...,-0.008454,-0.032839,0.004537,-0.020350,0.014824,0.054476,-0.007110,-0.021628,0.046494,-0.037668
3,0.055339,-0.016759,-0.039913,-0.020752,-0.004650,-0.020947,-0.025554,-0.003478,0.020020,-0.034456,...,-0.008453,-0.032840,0.004537,-0.020350,0.014824,0.054476,-0.007110,-0.021628,0.046494,-0.037668
4,0.055328,-0.016759,-0.039909,-0.020744,-0.004655,-0.020945,-0.025552,-0.003484,0.020014,-0.034450,...,-0.008452,-0.032841,0.004530,-0.020343,0.014825,0.054468,-0.007115,-0.021629,0.046495,-0.037670


In [34]:
with open(f"emb_PubMed_sample_{sample_no}.dat", "w") as file:
    file.write("\n")
    for idx, row in emb_df.iterrows():
        emb = row[:].astype(np.float32)
        emb_str = ' '.join(emb.astype(str))
        file.write(f'{idx}\t{emb_str}\n')
    